In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

from collections import Counter
import operator

In [2]:
filename = '../data/2019-Oct.csv_10%.csv'

In [3]:
import random
p = 1  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
         filename,
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)
df.shape

(4248118, 9)

# Preprocess

In [4]:
def preprocessing_feat(X, drop_event_time=False):
    if drop_event_time:
        X = X.drop("event_time", axis=1)
    
    X_preprocessed = X.dropna(subset = ['category_code', 'brand']) #tbd!!
    X_preprocessed = X_preprocessed.drop_duplicates()
    X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')
    return X_preprocessed

In [5]:
X_preprocessed = preprocessing_feat(df, drop_event_time=False)

/tmp/ipykernel_4548/4055017079.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')


In [6]:
X_preprocessed.shape

(2658228, 9)

In [7]:
def make_column(row):
    if row["price"] < row["25%"]:
        return "low"
    elif row["price"] < row["75%"]:
        return "medium"
    else:
        return "high"

def pricing_criterion(X):
    pricing_guide = X.groupby('category_code')['price'].describe()[["25%", "75%"]].reset_index()
    X_merged = X.merge(pricing_guide, on="category_code", how="right")
    X_merged["price_category"] = X_merged.apply(lambda row: make_column(row), axis=1)
    return X_merged


In [8]:
X_merged = pricing_criterion(X_preprocessed)

In [9]:
def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X


In [10]:
X_meta = metadata(X_merged)
X_meta.shape

(2658228, 13)

In [11]:
X_meta.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,25%,75%,price_category,metadata
0,2019-10-01 02:18:18 UTC,view,32900083,2055156924407612189,accessories bag,a-elita,8.49,549437633,62760980-f427-42e0-a891-8b590fd54584,16.555,76.96,low,accessories bag a-elita low
1,2019-10-01 02:38:43 UTC,view,21000027,2053013561420022511,accessories bag,trust,10.27,519029961,f1c690d1-b9ef-4a5b-b56f-0159f9198655,16.555,76.96,low,accessories bag trust low
2,2019-10-01 02:46:55 UTC,view,18300135,2053013558945383017,accessories bag,rivacase,28.29,542299866,053f1c21-370a-47f3-aa20-8bd1b27d7efe,16.555,76.96,medium,accessories bag rivacase medium
3,2019-10-01 02:48:59 UTC,view,28401176,2053013566209917945,accessories bag,karya,100.39,549348524,933c58ab-5185-44d6-94d6-c2ccb335d7f9,16.555,76.96,high,accessories bag karya high
4,2019-10-01 02:50:58 UTC,view,28400721,2053013566209917945,accessories bag,karya,61.26,547469497,50a09c8a-aa16-41d4-88c3-16808721ce13,16.555,76.96,medium,accessories bag karya medium


In [12]:
X_purch = X_meta[X_meta['event_type']=='purchase']

In [13]:
X_purch.shape

(54716, 13)

In [14]:
def filter(X):
    df = pd.DataFrame(X.groupby('user_session').event_type.count())
    df_new = df[df['event_type']>1]
    return df_new

In [15]:
X_filter = filter(X_purch)

In [16]:
X_filter.shape

(1088, 1)

In [17]:
X_filter.head()

,event_type
user_session,
004cbcc0-9763-49bd-a8a7-568e9003648f,2
00794112-3575-46b6-912b-42e8df0dc78b,2
0083e17d-90aa-4c03-a46e-ee71dea54f26,2
00848298-2ce1-4017-b488-cbe1ff454039,2
00848f4f-1fd4-4b7c-84ed-1dacc63bd352,2


In [18]:
df_cross = X_purch[X_purch['user_session'].isin(X_filter.index)]

In [19]:
df_cross.shape

(2268, 13)

# Counter

In [20]:
df_test = pd.DataFrame(df_cross.groupby('user_session')['product_id'].apply(list).tolist())
df_test.head()

,0,1,2,3,4,5,6,7,8
0,12400037,12400037,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4200545,3600952,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1004961,1201466,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306316,1306315,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1004249,1004249,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def concat(X):
    X = X[X[0]!=X[1]]
    X['conc'] = X[0].astype(str) + '_' + X[1].astype(str)
    
    lst = list(X['conc'])
    
    a = list(Counter(lst).keys()) # equals to list(X['conc'])
    b = list(Counter(lst).values()) # counts the elements' frequency
    
    dct_1 = {a[i]: b[i] for i in range(len(a))}
    dct_2 = dict(sorted(dct_1.items(), key=operator.itemgetter(1),reverse=True))

    return pd.DataFrame.from_dict(dct_2, orient='index').reset_index().rename(columns={'index': 'combo', 0: 'counts'})


In [22]:
X_sell = concat(df_test)

/tmp/ipykernel_4548/1906696682.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['conc'] = X[0].astype(str) + '_' + X[1].astype(str)


In [23]:
X_sell

,combo,counts
0,1004767_1004856,6
1,1004870_1004767,5
2,1004833_1004856,5
3,1004767_1004833,4
4,1004873_1004767,3
...,...,...
498,1004659_1004873,1
499,28719302_28715597,1
500,1004767_1004750,1
501,1002629_1004838,1


In [24]:
def x_seller(X):
    X["product_1"] = X["combo"].str.split("_",expand=True)[0].astype('category')
    X["product_2"] = X["combo"].str.split("_",expand=True)[1].astype('category')
    return X
    

In [25]:
new_dataframe = x_seller(X_sell)

new_dataframe.product_1 = new_dataframe.product_1.apply(lambda x: int(x))
new_dataframe.product_2 = new_dataframe.product_2.apply(lambda x: int(x))


In [26]:
new_dataframe.shape

(503, 4)

In [27]:
new_dataframe.head()

,combo,counts,product_1,product_2
0,1004767_1004856,6,1004767,1004856
1,1004870_1004767,5,1004870,1004767
2,1004833_1004856,5,1004833,1004856
3,1004767_1004833,4,1004767,1004833
4,1004873_1004767,3,1004873,1004767


In [28]:
new_df_1 = new_dataframe.merge(X_meta, how='left', left_on='product_1', right_on='product_id').rename(columns={'price': 'price_1', 'metadata': 'metadata_1'})

In [29]:
new_df_1.shape

(4192413, 17)

In [30]:
new_df_1 = new_df_1.drop_duplicates(['product_1','product_2'])
new_df_1.shape

(503, 17)

In [31]:
new_df_1.head()

,combo,counts,product_1,product_2,event_time,event_type,product_id,category_id,category_code,brand,price_1,user_id,user_session,25%,75%,price_category,metadata_1
0,1004767_1004856,6,1004767,1004856,2019-10-01 00:02:15 UTC,view,1004767,2053013555631882655,electronics smartphone,samsung,254.82,551417808,3f2209fb-0fe6-4093-8486-51e52dbda8cf,179.26,722.4,medium,electronics smartphone samsung medium
44056,1004870_1004767,5,1004870,1004767,2019-10-01 02:19:38 UTC,view,1004870,2053013555631882655,electronics smartphone,samsung,286.86,512740727,20e584bf-6461-4940-a774-1ff1110c633b,179.26,722.4,medium,electronics smartphone samsung medium
66074,1004833_1004856,5,1004833,1004856,2019-10-01 02:23:16 UTC,view,1004833,2053013555631882655,electronics smartphone,samsung,174.76,550422448,f9e45568-7f95-4367-8383-7dfdbd5c1791,179.26,722.4,low,electronics smartphone samsung low
89961,1004767_1004833,4,1004767,1004833,2019-10-01 00:02:15 UTC,view,1004767,2053013555631882655,electronics smartphone,samsung,254.82,551417808,3f2209fb-0fe6-4093-8486-51e52dbda8cf,179.26,722.4,medium,electronics smartphone samsung medium
134017,1004873_1004767,3,1004873,1004767,2019-10-01 02:20:27 UTC,view,1004873,2053013555631882655,electronics smartphone,samsung,388.81,512740727,20e584bf-6461-4940-a774-1ff1110c633b,179.26,722.4,medium,electronics smartphone samsung medium


In [32]:
df_short = new_df_1[['combo', 'counts', 'product_1', 'product_2', 'price_1', 'metadata_1']]
df_short.head()

,combo,counts,product_1,product_2,price_1,metadata_1
0,1004767_1004856,6,1004767,1004856,254.82,electronics smartphone samsung medium
44056,1004870_1004767,5,1004870,1004767,286.86,electronics smartphone samsung medium
66074,1004833_1004856,5,1004833,1004856,174.76,electronics smartphone samsung low
89961,1004767_1004833,4,1004767,1004833,254.82,electronics smartphone samsung medium
134017,1004873_1004767,3,1004873,1004767,388.81,electronics smartphone samsung medium


In [33]:
df_short.shape

(503, 6)

In [34]:
df_final = df_short.merge(X_meta, how='left', left_on='product_2', right_on='product_id').rename(columns={'price': 'price_2', 'metadata': 'metadata_2'}).drop_duplicates(['product_1','product_2'])

In [35]:
df_final.shape

(503, 19)

# Final recommendation

In [36]:
df_final = df_final[['combo', 'counts', 'product_1', 'product_2', 'price_1', 'price_2', 'metadata_1', 'metadata_2']]

In [37]:
df_final.shape

(503, 8)

In [38]:
df_final

,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2
0,1004767_1004856,6,1004767,1004856,254.82,130.76,electronics smartphone samsung medium,electronics smartphone samsung low
49537,1004870_1004767,5,1004870,1004767,286.86,254.82,electronics smartphone samsung medium,electronics smartphone samsung medium
93593,1004833_1004856,5,1004833,1004856,174.76,130.76,electronics smartphone samsung low,electronics smartphone samsung low
143130,1004767_1004833,4,1004767,1004833,254.82,174.76,electronics smartphone samsung medium,electronics smartphone samsung low
167017,1004873_1004767,3,1004873,1004767,388.81,254.82,electronics smartphone samsung medium,electronics smartphone samsung medium
...,...,...,...,...,...,...,...,...
5152668,1004659_1004873,1,1004659,1004873,787.18,388.81,electronics smartphone samsung high,electronics smartphone samsung medium
5168063,28719302_28715597,1,28719302,28715597,94.98,100.36,apparel shoes keds fila medium,apparel shoes keds fila medium
5168080,1004767_1004750,1,1004767,1004750,254.82,197.43,electronics smartphone samsung medium,electronics smartphone samsung medium
5180072,1002629_1004838,1,1002629,1004838,377.29,179.38,electronics smartphone apple medium,electronics smartphone oppo medium


In [39]:
df_final = df_final[df_final['metadata_1'] != df_final['metadata_2']]

In [40]:
df_final.head()

,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2
0,1004767_1004856,6,1004767,1004856,254.82,130.76,electronics smartphone samsung medium,electronics smartphone samsung low
143130,1004767_1004833,4,1004767,1004833,254.82,174.76,electronics smartphone samsung medium,electronics smartphone samsung low
211073,1002544_1004767,3,1002544,1004767,464.13,254.82,electronics smartphone apple medium,electronics smartphone samsung medium
255129,1004856_1004767,3,1004856,1004767,130.76,254.82,electronics smartphone samsung low,electronics smartphone samsung medium
322182,1002633_1004767,3,1002633,1004767,360.08,254.82,electronics smartphone apple medium,electronics smartphone samsung medium


In [41]:
df_final["1.1"] = df_final["metadata_1"].str.split(" ",expand=True)[0].astype('category')
df_final["1.2"] = df_final["metadata_1"].str.split(" ",expand=True)[1].astype('category')
df_final.head()

/tmp/ipykernel_4548/874055066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["1.1"] = df_final["metadata_1"].str.split(" ",expand=True)[0].astype('category')
/tmp/ipykernel_4548/874055066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["1.2"] = df_final["metadata_1"].str.split(" ",expand=True)[1].astype('category')


,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2,1.1,1.2
0,1004767_1004856,6,1004767,1004856,254.82,130.76,electronics smartphone samsung medium,electronics smartphone samsung low,electronics,smartphone
143130,1004767_1004833,4,1004767,1004833,254.82,174.76,electronics smartphone samsung medium,electronics smartphone samsung low,electronics,smartphone
211073,1002544_1004767,3,1002544,1004767,464.13,254.82,electronics smartphone apple medium,electronics smartphone samsung medium,electronics,smartphone
255129,1004856_1004767,3,1004856,1004767,130.76,254.82,electronics smartphone samsung low,electronics smartphone samsung medium,electronics,smartphone
322182,1002633_1004767,3,1002633,1004767,360.08,254.82,electronics smartphone apple medium,electronics smartphone samsung medium,electronics,smartphone


In [42]:
df_final["2.1"] = df_final["metadata_2"].str.split(" ",expand=True)[0].astype('category')
df_final["2.2"] = df_final["metadata_2"].str.split(" ",expand=True)[1].astype('category')
df_final.shape

/tmp/ipykernel_4548/2188218553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["2.1"] = df_final["metadata_2"].str.split(" ",expand=True)[0].astype('category')
/tmp/ipykernel_4548/2188218553.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["2.2"] = df_final["metadata_2"].str.split(" ",expand=True)[1].astype('category')


(341, 12)

In [43]:
df_final['test_1'] = df_final[['1.1', '1.2']].apply(lambda x: ' '.join(x), axis = 1)
df_final['test_2'] = df_final[['2.1', '2.2']].apply(lambda x: ' '.join(x), axis = 1)

/tmp/ipykernel_4548/1810607773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['test_1'] = df_final[['1.1', '1.2']].apply(lambda x: ' '.join(x), axis = 1)
/tmp/ipykernel_4548/1810607773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['test_2'] = df_final[['2.1', '2.2']].apply(lambda x: ' '.join(x), axis = 1)


In [44]:
df_final.head()

,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2,1.1,1.2,2.1,2.2,test_1,test_2
0,1004767_1004856,6,1004767,1004856,254.82,130.76,electronics smartphone samsung medium,electronics smartphone samsung low,electronics,smartphone,electronics,smartphone,electronics smartphone,electronics smartphone
143130,1004767_1004833,4,1004767,1004833,254.82,174.76,electronics smartphone samsung medium,electronics smartphone samsung low,electronics,smartphone,electronics,smartphone,electronics smartphone,electronics smartphone
211073,1002544_1004767,3,1002544,1004767,464.13,254.82,electronics smartphone apple medium,electronics smartphone samsung medium,electronics,smartphone,electronics,smartphone,electronics smartphone,electronics smartphone
255129,1004856_1004767,3,1004856,1004767,130.76,254.82,electronics smartphone samsung low,electronics smartphone samsung medium,electronics,smartphone,electronics,smartphone,electronics smartphone,electronics smartphone
322182,1002633_1004767,3,1002633,1004767,360.08,254.82,electronics smartphone apple medium,electronics smartphone samsung medium,electronics,smartphone,electronics,smartphone,electronics smartphone,electronics smartphone


In [45]:
df_final_2 = df_final[df_final['test_1'] != df_final['test_2']]

In [46]:
df_final_2.shape

(132, 14)

In [47]:
df_final_2.head()

,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2,1.1,1.2,2.1,2.2,test_1,test_2
763812,4804055_1004249,2,4804055,1004249,189.91,739.81,electronics audio headphone apple high,electronics smartphone apple high,electronics,audio,electronics,smartphone,electronics audio,electronics smartphone
903397,4200545_3600952,1,4200545,3600952,437.57,423.89,appliances environment air_conditioner elenber...,appliances kitchen washer samsung high,appliances,environment,appliances,kitchen,appliances environment,appliances kitchen
903534,1004961_1201466,1,1004961,1201466,179.36,237.27,electronics smartphone oppo medium,electronics tablet samsung medium,electronics,smartphone,electronics,tablet,electronics smartphone,electronics tablet
904269,5100722_1004767,1,5100722,1004767,169.62,254.82,electronics clocks huawei medium,electronics smartphone samsung medium,electronics,clocks,electronics,smartphone,electronics clocks,electronics smartphone
995651,1005160_11300053,1,1005160,11300053,231.41,17.25,electronics smartphone xiaomi medium,electronics telephone texet low,electronics,smartphone,electronics,telephone,electronics smartphone,electronics telephone


In [58]:
df_final_2.shape

(132, 14)

In [66]:
def X_seller(X, product_id, n=2):
    X.sort_values(by='counts')
    X = X.loc[X['product_1'] == product_id]
    X = X[['combo', 'counts', 'product_1', 'product_2', 'metadata_1', 'metadata_2', 'price_1', 'price_2']].iloc[:n,:]
    return X

In [68]:
X_seller(df_final_2, 5100722)

,combo,counts,product_1,product_2,metadata_1,metadata_2,price_1,price_2
904269,5100722_1004767,1,5100722,1004767,electronics clocks huawei medium,electronics smartphone samsung medium,169.62,254.82
